In [1]:
from datetime import datetime

from collections import namedtuple

import numpy as np
import pandas as pd

import sqlite3

In [2]:
sqlite3_path = './database/nanite_storage.sqlite3'

In [3]:
# defining dict-like structure for retrieved data (use ._asdict() method on them)
AuthorVals = namedtuple('Author', 'author_id username password acct_created_on')

In [7]:
def timestamp(): 
    return str(datetime.now())

In [161]:
def path_parser(path):
    """
    Takes in path, returns filetype
    """
    txt = '.txt'
    rtf = '.rtf'
    docx = '.docx'
    txt = path.rfind(txt)
    rtf = path.rfind(rtf)
    docx = path.rfind(docx)
    if txt != -1:
        return '.txt'
    elif rtf != -1:
        return '.rtf'  
    elif docx != -1:
        return '.docx'   
    else:
        print("Incompatible Filetype.")   

In [27]:
def create_author(author_name, password=''):    
    """ 
    Inserts an Author into database if they don't exist.
    If they do, this task does not go through. 
    """
    # add a check to make sure the author doesn't already exist
    now = timestamp()
    conn = sqlite3.connect(sqlite3_path)
    cur = conn.cursor()
    
    cur.execute("SELECT username FROM authors WHERE username=?", (author_name,))
    entry = cur.fetchone()
    if entry is None:

        query = '''INSERT INTO authors
                    values (?,?,?,?) '''
        
        params = (None, author_name, password, now)
        cur.execute(query, params)
        conn.commit()
        conn.close()
    else:
        conn.close()
        return print("This row already exists!")

In [137]:
def return_author_list():
    """
    Prints all authors as a list of dicts
    """
    conn = sqlite3.connect(sqlite3_path)
    cur = conn.cursor()
    cur.execute("SELECT * FROM authors")
    rows = cur.fetchall()
    data = [dict(AuthorVals(*rows[i])._asdict()) for i in range(len(rows))]
    conn.close()
    return data

In [ ]:
# create a function to infer some class-related details

In [8]:
class AuthorActions:

    def __init__(self, author_id=-1):
        self.author_id = author_id
    

    def return_info(self):
        """
        prints the row of information for the author
        """ 
        conn = sqlite3.connect(sqlite3_path)
        cur = conn.cursor()
        cur.execute("SELECT * FROM authors WHERE author_id=?", (self.author_id,))
        row = cur.fetchall()
        data = AuthorVals(*row[0])
        conn.close()
        return dict(data._asdict())


    def id_by_name(self, author_name):
        """
        Sets AuthorChecker to the right ID based on the author's name.
        """
        conn = sqlite3.connect(sqlite3_path)
        cur = conn.cursor()
        cur.execute("SELECT * FROM authors WHERE username=?", (author_name,))
        data = cur.fetchone()
        if data is not None:
            self.author_id = int(data[0])
            conn.close()
        else:
            print("No name found.")
            conn.close()


    def create_project(
        self, 
        project_name, 
        deadline=None,
        wordcount_goal=None, 
        current_daily_target=None,
        filetype='na',
        is_folder=True,
        project_path=None):
        """
        Takes in a project info, then creates a new project for a given author.
        Eric note: I may switch this to taking in a dict in future, depending on Bens feedback.
        """
        now = timestamp()
        conn = sqlite3.connect(sqlite3_path)
        cur = conn.cursor()
        cur.execute("SELECT project_id FROM projects WHERE project_name=? AND author_id=?", (project_name,self.author_id))
        entry = cur.fetchone()
        conn.close()
        if entry is None:
            # do insert
            conn = sqlite3.connect(sqlite3_path)
            cur = conn.cursor()
            query = '''INSERT INTO projects values (
                ?, ?, ?, ?, ?, ?, ?, ?, ?, ?) '''
            params = [None, self.author_id, project_name, now, deadline, 
            wordcount_goal, current_daily_target, filetype, is_folder, project_path]
            try:
                cur.execute(query, params)
                conn.commit()
            finally:
                conn.close()
        else:
            print("project with this name already exists")
            conn.close()


    def delete_project(self, project_name):
        conn = sqlite3.connect(sqlite3_path)
        cur = conn.cursor()
        cur.execute("DELETE FROM projects where project_name=? AND author_id=?", (project_name, self.author_id))
        conn.commit()
        conn.close()


    def return_projects(self):
        conn = sqlite3.connect(sqlite3_path)
        cur = conn.cursor()
        cur.execute("SELECT * FROM projects where author_id=?", self.author_id)
        data = cur.fetchall()
        conn.close()
        return data


    def rename_author(self, new_name):
        """Renames an author based on new_name"""
        conn = sqlite3.connect(sqlite3_path)
        cur = conn.cursor()
        cur.execute("UPDATE authors SET username=? WHERE author_id=?", (new_name, self.author_id))
        conn.commit()
        conn.close()

In [ ]:
class ProjectActions:
    
    def __init__(self, project_id=-1, author_id=-1):
        self.project_id = project_id
        self.author_id = author_id

    def set_current_project(self, author_id, project_id):
        # check author id & project id in the database- reset 
    
    def find_name_from_project_id(self):
        


    def return_author(self):
        # set self

In [8]:
#code.define_author("george")
code.author_id

3

In [5]:

### DDL Queries ###

create_author_sql = """
CREATE TABLE IF NOT EXISTS authors (
    author_id integer PRIMARY KEY,
    username text NOT NULL,
    password text,
    acct_created_on text NOT NULL
)
"""

create_project_sql = """
CREATE TABLE IF NOT EXISTS projects (
    project_id integer PRIMARY KEY,
    author_id integer,
    project_name text NOT NULL,
    project_created_on text NOT NULL,
    deadline text,
    wordcount_goal int,
    current_daily_target int,
    filetype text NOT NULL,
    is_folder int NOT NULL,
    project_path text NOT NULL,
    FOREIGN KEY (author_id) REFERENCES authors (author_id)
)
"""

create_words_sql = """
CREATE TABLE IF NOT EXISTS words (
    record_id integer PRIMARY_KEY,
    project_id integer,
    author_id integer,
    Wdate text NOT NULL,
    Wcount int NOT NULL,
    Wtarget int,
    FOREIGN KEY (project_id) REFERENCES projects (project_id),
    FOREIGN KEY (author_id) REFERENCES authors (author_id)
)
"""


sqlite3_path = './database/nanite_storage.sqlite3'
# SQLite just needs to connect to create db
def db_init():
    """creates database if they don't exist"""
    if os.path.exists(sqlite3_path) == False:
        conn = sqlite3.connect(sqlite3_path)
        conn.execute("PRAGMA foreign_keys = 1")
        conn.execute(create_author_sql)
        conn.execute(create_project_sql)
        conn.execute(create_words_sql)
        conn.close()
    else:
        print("db already exists")

In [6]:
db_init()

In [9]:
a = AuthorActions()
a.id_by_name('Johnny')
a.return_info()


{'author_id': 1,
 'username': 'Johnny',
 'password': '',
 'acct_created_on': '2021-04-02 18:02:02.682807'}

In [11]:
a.create_project("Johnnys Manifesto", project_path='')

OperationalError: database is locked

NameError: name 'conn' is not defined